In [119]:
import requests
from bs4 import BeautifulSoup
import re
import time
import json
from lxml import html

In [1]:
states = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming']

In [3]:
url = 'https://militarybenefits.info/bah-rates-state/{}/'

In [14]:
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36',
    'cookie': '_ga=GA1.2.308726539.1576501840; trc_cookie_storage=taboola%2520global%253Auser-id%3D9a63df74-cb51-41df-90f2-2fd17fd08cd3-tuct2f8393c; _cmpQcif3pcsupported=1; googlepersonalization=Orqxc-Orqxc-gA; _gid=GA1.2.2040505567.1576590923; __atuvc=10%7C51; __atuvs=5df8de4a1aaa54e4003; _gat_UA-864394-17=1',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}


In [15]:
soup = BeautifulSoup(resp.text)

In [68]:
# soup.find('h2', class_ = re.compile(r'.*follow\?page.*'))
soup.findAll('h2', {'class': re.compile(r'tablepress-table-name tablepress-table-name-id-.{,4}')})

[<h2 class="tablepress-table-name tablepress-table-name-id-65">MHA: CO045 - Military Housing Area Name: DENVER, CO</h2>,
 <h2 class="tablepress-table-name tablepress-table-name-id-66">MHA: CO046 - Military Housing Area Name: COLORADO SPRINGS, CO</h2>,
 <h2 class="tablepress-table-name tablepress-table-name-id-67">MHA: CO047 - Military Housing Area Name: FORT COLLINS, CO</h2>,
 <h2 class="tablepress-table-name tablepress-table-name-id-68">MHA: CO422 - Military Housing Area Name: BOULDER, CO</h2>]

In [52]:
soup.findAll('tr', {'class': 'row-6 even'})

[<tr class="row-6 even">
 <td class="column-1">E05</td><td class="column-2"> $2,208 </td><td class="column-3"> $1,761 </td>
 </tr>, <tr class="row-6 even">
 <td class="column-1">E05</td><td class="column-2"> $1,839 </td><td class="column-3"> $1,590 </td>
 </tr>, <tr class="row-6 even">
 <td class="column-1">E05</td><td class="column-2"> $1,740 </td><td class="column-3"> $1,530 </td>
 </tr>, <tr class="row-6 even">
 <td class="column-1">E05</td><td class="column-2"> $2,010 </td><td class="column-3"> $1,617 </td>
 </tr>]

In [56]:
tr = html.fromstring(resp.text)

In [102]:
a=[int(a.replace('$','').replace(',','')) for a in tr.xpath("//tbody[@class='row-hover']/tr[@class='row-6 even']/td[@class='column-2']/text()")]

In [103]:
b=[re.findall(r'MHA.*Name: (.*)',a)[0] for a in tr.xpath("//h2[re:match(@class, 'tablepress-table-name tablepress-table-name-id-.{,4}')]/text()",namespaces={'re':'http://exslt.org/regular-expressions'})]

In [105]:
list(zip(b,a))

[('DENVER, CO', 2208),
 ('COLORADO SPRINGS, CO', 1839),
 ('FORT COLLINS, CO', 1740),
 ('BOULDER, CO', 2010)]

In [114]:
bah = []
for state in states:
    resp = requests.get(url.format(state.lower()), headers = headers)
    tr = html.fromstring(resp.text)
    locations = [re.findall(r'MHA.*Name: (.*)',a)[0] for a in tr.xpath("//h2[re:match(@class, 'tablepress-table-name tablepress-table-name-id-.{,4}')]/text()",namespaces={'re':'http://exslt.org/regular-expressions'})]
    prices = a=[int(a.replace('$','').replace(',','')) for a in tr.xpath("//tbody[@class='row-hover']/tr[@class='row-6 even']/td[@class='column-2']/text()")]
    for a in zip(locations, prices):
        bah.append(a)
    time.sleep(2)

In [120]:
bah_dict = {}
for b in bah:
    bah_dict[b[0]] = b[1] 
with open('C:/Users/tamas.hegedus/Desktop/bah_data.json', 'w+') as f:
    json.dump(bah_dict, f)